In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
#pip install yfinance
from datetime import datetime
from talib import RSI, SMA, EMA, ADX, MACD, MFI, BBANDS, STOCH,MA_Type

In [4]:
url = "https://raw.githubusercontent.com/JacksonvBarbosa/Analise_Indice_Ibovespa/refs/heads/Jackson/Dataset/Dados%20Hist%C3%B3ricos%20-%20Ibovespa.csv"

In [5]:
df_ibov = pd.read_csv(url)

In [6]:
df_ibov_copia = df_ibov.copy()
df_ibov_copia

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,04.06.2025,137.002,137.547,138.797,136.695,"10,20M","-0,40%"
1,03.06.2025,137.546,136.787,137.672,136.175,"9,70B","0,56%"
2,02.06.2025,136.787,137.026,138.471,136.483,"9,10B","-0,18%"
3,30.05.2025,137.027,138.546,138.637,136.726,"15,14B","-1,09%"
4,29.05.2025,138.534,138.869,139.108,137.993,"8,83B","-0,25%"
...,...,...,...,...,...,...,...
597,10.01.2023,110.817,109.129,111.193,108.478,"13,70M","1,55%"
598,09.01.2023,109.130,108.964,109.938,108.134,"12,05M","0,15%"
599,06.01.2023,108.964,107.642,109.433,107.642,"12,63M","1,23%"
600,05.01.2023,107.641,105.336,107.743,105.333,"15,51M","2,19%"


In [7]:
# Formatando caracteristica Data e Convertendo em datetime
#df_ibov_copia['Data'] = pd.to_datetime(df_ibov_copia['Data'], format = '%d.%m.%Y').dt.strftime('%d/%m/%Y')
#df_ibov_copia['Data'] = pd.to_datetime(df_ibov_copia['Data'], format='%d.%m.%Y')

# Criando index tipo Dateindex
dateparse = lambda dates: datetime.strptime(dates, '%d.%m.%Y')
df_ibov_copia = pd.read_csv(url, parse_dates = ['Data'], index_col= 'Data', date_parser=dateparse)

df_ibov_copia

C:\Users\jacom\AppData\Local\Temp\ipykernel_21044\333086338.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_ibov_copia = pd.read_csv(url, parse_dates = ['Data'], index_col= 'Data', date_parser=dateparse)


,Último,Abertura,Máxima,Mínima,Vol.,Var%
Data,,,,,,
2025-06-04,137.002,137.547,138.797,136.695,"10,20M","-0,40%"
2025-06-03,137.546,136.787,137.672,136.175,"9,70B","0,56%"
2025-06-02,136.787,137.026,138.471,136.483,"9,10B","-0,18%"
2025-05-30,137.027,138.546,138.637,136.726,"15,14B","-1,09%"
2025-05-29,138.534,138.869,139.108,137.993,"8,83B","-0,25%"
...,...,...,...,...,...,...
2023-01-10,110.817,109.129,111.193,108.478,"13,70M","1,55%"
2023-01-09,109.130,108.964,109.938,108.134,"12,05M","0,15%"
2023-01-06,108.964,107.642,109.433,107.642,"12,63M","1,23%"


In [8]:
# Função para conversão dos dados de Volume de string para float
## OBS.: Caso acertando com o grupo e vermos a necessidade de mudar par outro tipo fazer a mudança
def converter_volume(vol):
    if pd.isna(vol):
        return np.nan
    vol = vol.upper().replace(',', '').strip()
    if vol.endswith('B'):
        return float(vol[:-1]) * 1e9
    elif vol.endswith('M'):
        return float(vol[:-1]) * 1e9
    elif vol.endswith('K'):
        return float(vol[:-1]) * 1e3
    else:
        return float(vol)
        
df_ibov_copia['Vol.'] = df_ibov_copia['Vol.'].apply(converter_volume)

In [9]:
# Substitui os caracteres (%) por (vázio) e (,) por (.) e converte tipo objeto para float
df_ibov_copia['Var%'] = df_ibov_copia['Var%'].str.replace('%', '').str.replace(',', '.').astype(float)

In [10]:
# Renomeando nome da Coluna Último para Fechamento_Diario
df_ibov_copia = df_ibov_copia.rename(columns={"Último": "Fechamento_Diario"})

In [11]:
# Cria a coluna fechamento com valores Binarios 0 para negativo e 1 para positivo
df_ibov_copia['Tendencia'] = (df_ibov_copia['Fechamento_Diario'].shift(-1) < df_ibov_copia['Fechamento_Diario']).astype('Int64')

# Indicadores para teste

In [17]:
print('RSI - ')
display(RSI(df_ibov_copia['Fechamento_Diario'], timeperiod=14))

print('SMA - Média Móvel Simples')
display(SMA(df_ibov_copia['Fechamento_Diario'], timeperiod=17))

print('Média Móvel Exponêncial')
display(EMA(df_ibov_copia['Fechamento_Diario'], timeperiod=17))

print('Bandas de Bollinger')
upperband, middleband, lowerband = BBANDS(df_ibov_copia['Fechamento_Diario'], matype= MA_Type.T3)
display(upperband) # Inserir Banda Superior
display(middleband)
display(lowerband) # Inserir Banda Inferior

# ADX - Average Directional Index
print("ADX")
display(ADX(df_ibov_copia['Máxima'], df_ibov_copia['Mínima'], df_ibov_copia['Fechamento_Diario'], timeperiod=17))

# MACD
print("MACD")
macd, macdsignal, macdhist = MACD(df_ibov_copia['Fechamento_Diario'], fastperiod=12, slowperiod=26, signalperiod=9)
display(macd) # Inserir esses dados para buscar valores acima e abaixo de 0
display(macdsignal)
display(macdhist) # Inserir esses dados para buscar valores acima e abaixo de 0

# MFI - Indice de Fluxo de Dinheiro
print("MFI")
display(MFI(df_ibov_copia['Máxima'], df_ibov_copia['Mínima'], df_ibov_copia['Fechamento_Diario'], df_ibov_copia['Vol.'], timeperiod=14))

# Estocastico
print("stoch - Estocatisco")
slowk, slowd = STOCH(df_ibov_copia['Máxima'], df_ibov_copia['Mínima'], df_ibov_copia['Fechamento_Diario'], fastk_period=5, slowk_period=14, slowk_matype=0, slowd_period=3, slowd_matype=0)
display(slowk)
display(slowd)

RSI - 


Data
2025-06-04          NaN
2025-06-03          NaN
2025-06-02          NaN
2025-05-30          NaN
2025-05-29          NaN
                ...    
2023-01-10    50.914197
2023-01-09    45.556597
2023-01-06    45.054203
2023-01-05    41.158473
2023-01-04    35.408868
Length: 602, dtype: float64

SMA - Média Móvel Simples


Data
2025-06-04           NaN
2025-06-03           NaN
2025-06-02           NaN
2025-05-30           NaN
2025-05-29           NaN
                 ...    
2023-01-10    112.191176
2023-01-09    112.018000
2023-01-06    111.755235
2023-01-05    111.482765
2023-01-04    111.072059
Length: 602, dtype: float64

Média Móvel Exponêncial


...

MFI


Data
2025-06-04          NaN
2025-06-03          NaN
2025-06-02          NaN
2025-05-30          NaN
2025-05-29          NaN
                ...    
2023-01-10    45.662813
2023-01-09    39.938315
2023-01-06    34.034963
2023-01-05    33.482460
2023-01-04    33.161245
Length: 602, dtype: float64

stoch - Estocatisco


Data
2025-06-04          NaN
2025-06-03          NaN
2025-06-02          NaN
2025-05-30          NaN
2025-05-29          NaN
                ...    
2023-01-10    50.086128
2023-01-09    47.381246
2023-01-06    43.102267
2023-01-05    39.583389
2023-01-04    37.909981
Length: 602, dtype: float64

Data
2025-06-04          NaN
2025-06-03          NaN
2025-06-02          NaN
2025-05-30          NaN
2025-05-29          NaN
                ...    
2023-01-10    51.461223
2023-01-09    49.754882
2023-01-06    46.856547
2023-01-05    43.355634
2023-01-04    40.198546
Length: 602, dtype: float64